#######

wide & deep
重新改写api为core

结果：仅能用于做预测，没有解释性

#######

In [23]:
import tensorflow as tf
import pandas as pd
import tempfile

In [2]:
# import data
df_all = pd.read_csv('data/all.csv').drop(['Unnamed: 0'], axis=1)
df_part = pd.concat([df_all.iloc[:, :23], df_all.iloc[:, 71:]], axis=1)

### tf core的特征列表

![](img/feature.png)

tf的特征主要分为3种：类别型、连续型、混合型


1. tf.feature_column.numerical_column——>对应tf.contrib.layers.real_valued_column: 数值列(连续型)
2. tf.feature_column.bucketized_column——>对应tf.contrib.layers.bucketized_column: 分桶列（连续——>离散），分桶列中相当于每个桶中对应了一系列值
3. tf.feature_column.categorical_column_with_identify——>对应tf.contrib.layers.sparse_column_with_integrized_feature: 分类标识列。可以视为分桶列的特殊情况。分桶列中相当于每个桶中对应了一系列值，分类标识列相当于一个桶仅仅对应一种情况
4. tf.feature_column.categorical_column_with_vocabulary_list——>对应tf.contrib.layers.sparse_column_with_keys: 分类词汇列(面向枚举的值不是很多的)。字符串表示为one-hot的方法
5. tf.feature_column.categorical_column_with_vocabulary_file——>对应tf.contrib.layers.sparse_column_with_files: 分类词汇列（面向需要枚举的值较多的特征）。导入的file，应该让每一个词汇各占一行。
6. tf.feature_column.categorical_column_with_hash_bucket——>对应tf.contrib.layers.sparse_column_with_hash_bucket: 哈希分桶列。通常我们的处理类别比较少，但有时候类别较多，如果为每一个类都设置相同的类别，会消耗较多内存。使用此方法会计算类别的hash值，再用模运算符将其置于不同的bucket中。直觉上会觉得，一些完全不相关的类会被放到同一个bucket中，但实际在计算中hash类别提供了一些分割方式，模型可以使用其它特征列进一步将两个不相关的类分开
7. tf.feature_column.indicator_column——>特征列。multi hot的表示方式
8. tf.feature_column.embedding_column——>对应tf.contrib.layers.embedding_column: 嵌入列，将数据表示为低维度的矢量。

In [3]:
# categorical base column(5)——离散类别列
visa_typ_cd = tf.feature_column.categorical_column_with_vocabulary_list(
    key="visa_typ_cd", 
    vocabulary_list=["A", "A1", "A2", "A3", "B", "C", "D1", "D2", "D3"])
top_schl_id = tf.feature_column.categorical_column_with_vocabulary_list(
    key="top_schl_id", 
    vocabulary_list=["432466.0", "430552.0", "431016.0", "432643.0", "434410.0",
       "432715.0", "434200.0", "409.0", "432710.0", "430564.0",
       "432699.0", "436223.0", "434266.0", "437282.0", "432468.0",
       "439174.0", "435391.0", "432590.0", "432791.0", "437357.0",
       "438276.0", "436224.0", "435620.0", "430558.0", "432610.0",
       "439177.0", "437353.0", "438456.0", "430627.0", "439175.0",
       "437317.0", "437367.0", "438445.0", "435300.0", "436070.0",
       "437414.0", "435590.0", "439176.0", "435467.0", "440467.0"])
year = tf.feature_column.categorical_column_with_vocabulary_list(
    key="year", 
    vocabulary_list=["2013", "2014", "2015", "2016", "2017", "2018"])
city = tf.feature_column.categorical_column_with_vocabulary_list(
    key="city", 
    vocabulary_list=["changchun", "shanghai", "beijin", "zhuhai", "yantai", "changsha",
        "chongqing", "yangzhou", "linfen", "taizhou", "wenzhou", "wuhan", "jiamusi", 
        "nantong", "wuxi", "hangzhou", "dalian", "huzhou",
        "shenzheng", "xian", "ningbo", "changzhou"])
level = tf.feature_column.categorical_column_with_vocabulary_list(
    key="level", 
    vocabulary_list=["2nd", "1st", "new1", "4th", "3rd"])

In [4]:
# continues column(27)——数值列
par_days = tf.feature_column.numeric_column("par_days")
actual_day = tf.feature_column.numeric_column("actual_day")
sub_tol = tf.feature_column.numeric_column("sub_tol")
sub_avg = tf.feature_column.numeric_column("sub_avg")
sub_avg_a = tf.feature_column.numeric_column("sub_avg_a")
ex_tol = tf.feature_column.numeric_column("ex_tol")
ex_subrate = tf.feature_column.numeric_column("ex_subrate")
sub_ins_sd = tf.feature_column.numeric_column("sub_ins_sd")
sub_ins_m = tf.feature_column.numeric_column("sub_ins_m")
ins_m = tf.feature_column.numeric_column("ins_m")
auc_sub_avg = tf.feature_column.numeric_column("auc_sub_avg")
auc_sub_sd = tf.feature_column.numeric_column("auc_sub_sd")
ins_max = tf.feature_column.numeric_column("ins_max")
ins_summit_four_1 = tf.feature_column.numeric_column("ins_summit_four_1")
ins_summit_four_2 = tf.feature_column.numeric_column("ins_summit_four_2")
ins_summit_four_3 = tf.feature_column.numeric_column("ins_summit_four_3")
ins_summit_max = tf.feature_column.numeric_column("ins_summit_max")
notscWe_sub = tf.feature_column.numeric_column("notscWe_sub")
notscWe_ex = tf.feature_column.numeric_column("notscWe_ex")
notscWd_sub = tf.feature_column.numeric_column("notscWd_sub")
notscWd_ex = tf.feature_column.numeric_column("notscWd_ex")
sc_sub = tf.feature_column.numeric_column("sc_sub")
sc_ex = tf.feature_column.numeric_column("sc_ex")
pub_avg = tf.feature_column.numeric_column("pub_avg")
pub_sd = tf.feature_column.numeric_column("pub_sd")
n_ex = tf.feature_column.numeric_column("n_ex")
n_pkg = tf.feature_column.numeric_column("n_pkg")
n_pra_ex_avg = tf.feature_column.numeric_column("n_pra_ex_avg")
n_pra_ex_sd = tf.feature_column.numeric_column("n_pra_ex_sd")
delta_days = tf.feature_column.numeric_column("delta_days")

In [5]:
# bucketized categorical column(3)——分桶列
par_days_buckets = tf.feature_column.bucketized_column(
    par_days, 
    boundaries=[7, 31, 93, 183, 365, 730])
actual_day_buckets = tf.feature_column.bucketized_column(
    actual_day, 
    boundaries=[5, 25, 75, 150, 300, 600])
delta_days_buckets = tf.feature_column.bucketized_column(
    delta_days, 
    boundaries=[7, 31, 93, 183, 365, 730, 1650])

In [6]:
# define wide features
wide_columns = [
    visa_typ_cd, 
    city,
    top_schl_id, year, level, par_days_buckets, 
    actual_day_buckets, delta_days_buckets,
    tf.feature_column.crossed_column([visa_typ_cd, top_schl_id], hash_bucket_size=int(1e4)),
    tf.feature_column.crossed_column([visa_typ_cd, year], hash_bucket_size=int(1e4)),
    tf.feature_column.crossed_column([visa_typ_cd, city], hash_bucket_size=int(1e4)),
    tf.feature_column.crossed_column([visa_typ_cd, level], hash_bucket_size=int(1e4)),
    tf.feature_column.crossed_column([visa_typ_cd, top_schl_id, year], hash_bucket_size=int(1e4)),
    tf.feature_column.crossed_column([year, top_schl_id], hash_bucket_size=int(1e4))    
]

In [7]:
# define deep features
# embedding的维度越高，这个模型不得不学习的特征表示的自由度越大。基于经验，一般选择log2(n) 【以2为底】,n是一个特征列中唯一特征的数量
deep_columns = [
   par_days, actual_day, sub_tol, sub_avg, sub_avg_a, ex_tol, ex_subrate, sub_ins_sd, 
   sub_ins_m, ins_m, auc_sub_avg, auc_sub_sd, ins_max, ins_summit_four_1, ins_summit_four_2,
   ins_summit_four_3, ins_summit_max, notscWe_sub, notscWe_ex, notscWd_sub, notscWd_ex, 
   sc_sub, sc_ex, pub_avg, pub_sd, n_ex, n_pkg, n_pra_ex_avg, n_pra_ex_sd, delta_days,
   tf.feature_column.embedding_column(visa_typ_cd, dimension=3),
   tf.feature_column.embedding_column(top_schl_id, dimension=5),
   tf.feature_column.embedding_column(year, dimension=3),
   tf.feature_column.embedding_column(city, dimension=5),
   tf.feature_column.embedding_column(level, dimension=3)
]

In [9]:
# 创建临时文件存储过程数据
model_dir = tempfile.mkdtemp() # create a temp path
m = tf.estimator.DNNLinearCombinedClassifier(
    model_dir=model_dir, 
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100,60,30]   
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x108b0b1d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmp95brnh', '_save_summary_steps': 100}


In [10]:
# define the column names for datasets
COLUMNS = [
        'visa_typ_cd', 
        'city'
        'par_days', 'actual_day', 'sub_tol',
       'sub_avg', 'sub_avg_a', 'ex_tol', 'ex_subrate', 'sub_ins_sd',
       'sub_ins_m', 'ins_m', 'auc_sub_avg', 'auc_sub_sd', 'ins_max',
       'ins_summit_four_1', 'ins_summit_four_2', 'ins_summit_four_3',
       'ins_summit_max', 'notscWe_sub', 'notscWe_ex', 'notscWd_sub',
       'notscWd_ex', 'sc_sub', 'sc_ex', 'pub_avg', 'pub_sd', 'n_ex',
       'n_pkg', 'n_pra_ex_avg', 'n_pra_ex_sd', 'top_schl_id',
       'year','level','delta_days']

LABEL_COLUMN = 'y'

CATEGORY_COLUMNS = [
    'visa_typ_cd',
    'city',
    'top_schl_id','year','level']

CONTINUES_COLUMNS = ['par_days', 'actual_day', 'sub_tol',
       'sub_avg','sub_avg_a', 'ex_tol', 'ex_subrate', 'sub_ins_sd',
       'sub_ins_m', 'ins_m', 'auc_sub_avg', 'auc_sub_sd', 'ins_max',
       'ins_summit_four_1', 'ins_summit_four_2', 'ins_summit_four_3',
       'ins_summit_max', 'notscWe_sub', 'notscWe_ex', 'notscWd_sub',
       'notscWd_ex', 'sc_sub', 'sc_ex', 'pub_avg', 'pub_sd', 'n_ex',
       'n_pkg', 'n_pra_ex_avg', 'n_pra_ex_sd','delta_days']


# shuffle the dataset
df_test = df_part.sample(frac=0.008)
# choose the test dataset in the left & shuffle
df_train = df_part[[False if i in df_test.index else True for i in range(len(df_part))]].sample(frac=0.024)

test_label = df_part[[True if i in df_test.index.values else False for i in df_part.index.values]].y.values
test_label = [int(label) for label in test_label]
train_label = df_part[[True if i in df_train.index.values else False for i in df_part.index.values]].y.values
train_label = [int(label) for label in train_label]
df_test[LABEL_COLUMN] = test_label
df_train[LABEL_COLUMN] = train_label
df_train = df_train.reset_index().drop(['index'], axis=1)
df_test = df_test.reset_index().drop(['index'], axis=1)
df_train['top_schl_id'] = [str(val) for val in df_train.top_schl_id.values]
df_test['top_schl_id'] = [str(val) for val in df_test.top_schl_id.values]
df_train['year'] = [str(val) for val in df_train.year.values]
df_test['year'] = [str(val) for val in df_test.year.values]


def input_fn(df):
    # create a dictonary mapping from each continues features column name(k) 
    # to the values of that column stored in a constant tensor
    continuous_cols = {k: tf.constant(df[k].values) 
                       for k in CONTINUES_COLUMNS}
    
    # create a dictonary mapping from each catrgorical features column name(k) 
    # to the values of that column stored in a constant tensor
    categorical_cols = {
        k: tf.SparseTensor(
            indices = [[i,0] for i in range(df[k].size)],
            values = df[k].values,
            dense_shape = [df[k].size, 1]
        )
        for k in CATEGORY_COLUMNS
    }
    
    # merge the two dictionaries into one
    feature_cols = dict(continuous_cols.items() + categorical_cols.items())
    # converts the labels column into a constant tensor
    label = tf.constant(df[LABEL_COLUMN].values)
    # return the feature columns and the label
    return feature_cols, label   


def train_input_fn():
    return input_fn(df_train)

def test_input_fn():
    return input_fn(df_test)    

In [11]:
m.train(input_fn = train_input_fn,steps=200)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmp95brnh/model.ckpt.
INFO:tensorflow:loss = 1215502.2, step = 1
INFO:tensorflow:global_step/sec: 10.6827
INFO:tensorflow:loss = 51373.297, step = 101 (9.361 sec)
INFO:tensorflow:Saving checkpoints for 200 into /var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmp95brnh/model.ckpt.
INFO:tensorflow:Loss for final step: 44748.59.


In [12]:
results = m.evaluate(input_fn=test_input_fn,steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-14-02:35:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmp95brnh/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-06-14-02:35:30
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.70581394, accuracy_baseline = 0.88546515, auc = 0.49771982, auc_precision_recall = 0.9182372, average_loss = 3.5634959, global_step = 200, label/mean = 0.88546515, loss = 12258.426, prediction/mean = 0.76566136


In [12]:
results

{'accuracy': 0.8241279,
 'accuracy_baseline': 0.88313955,
 'auc': 0.51459455,
 'auc_precision_recall': 0.9368909,
 'average_loss': 7.70956,
 'global_step': 200,
 'label/mean': 0.88313955,
 'loss': 26520.887,
 'prediction/mean': 0.91530037}

auc不适用imblanced的情况，相反pr_auc更适合imbalance下的评价